This notebook is an (in progress) example of kernel optimization using SVI in gp3. Use NBViewer to view the notebook [here](https://nbviewer.jupyter.org/github/as4529/gp3/blob/master/examples/kernel_learning.ipynb?flush_cache=true). 

In [1]:
from gp3.inference import MFSVI
from gp3.likelihoods import Poisson, Bernoulli
from gp3.utils import data as sim
from gp3.kernels import RBF
from gp3.utils.transforms import softplus, inv_softplus
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
from IPython.display import display
init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from tqdm import trange
from ipywidgets import IntProgress
import numpy as np
from scipy.special import expit

First, we simulate some data from a GP with an RBF kernel.

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 20, lower=0, upper=100)
f = sim.sim_f(X, RBF(30., 3., .1), mu = 0.)
y = sim.poisson_draw(f, 1.) 

trace_func = go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))
trace_draws = go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))
fig = tools.make_subplots(rows=1, cols=2, specs=[[{'is_3d': True}, {'is_3d': True}]])
fig.append_trace(trace_func, 1, 1)
fig.append_trace(trace_draws, 1, 2)
iplot(fig)

This is the format of your plot grid:
[ (1,1) scene1 ]  [ (1,2) scene2 ]



Now, we set the "opt_kernel" parameter for the mean field SVI inference to True, so that we optimize over our base kernel hyperparameters as we optimize the ELBO. This is currently very sensitive to initialization. Addressing this sensitivity is my current priority. Also note that (as expected) there is around a 5x slowdown when we are learning the kernel as opposed to keeping it fixed.

In [3]:
inf_svi = MFSVI(X, y, RBF(10., 1., .1), Poisson(), opt_kernel = True)
inf_svi.run(2000)

## Results

Plotting posterior mean and variances

In [4]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=inf_svi.predict(), mode = 'markers', marker=dict(size = 2,))])
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=np.exp(inf_svi.q_S), mode = 'markers', marker=dict(size = 2,))])

In [5]:
iplot([go.Scatter(x = list(range(len(inf_svi.elbos))), y = inf_svi.elbos)])

In [6]:
softplus(inf_svi.kernel.params)

array([44.24007554, 11.55761263,  0.1       ])